In [1]:
import os
import time
import numpy as np
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers.convolutional import Conv1D
from sklearn import preprocessing
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from numpy.fft import fft, fftfreq, ifft
import scipy
from scipy.fftpack import fft
from scipy.fftpack import dct
from scipy import fftpack
from scipy import signal
import csv
from scipy.signal import filtfilt
from scipy.signal import lfilter
from scipy import stats
from scipy.signal import butter, lfilter, lfilter_zi, normalize


In [2]:
Channel = ['AF3', 'F7', 'F3', 'FC5', 'T7', 'P7', 'O1', 'O2', 'P8', 'T8', 'FC6', 'F4', 'F8', 'AF4']

In [3]:
def bandPassFilter(y, low, high):
    lowcut = low
    highcut = high
    nSamples = len(y)
    signal = y
    order = 2
    fs = 256
    nyq = 0.5 * fs
    l = lowcut / nyq
    h = highcut / nyq
    b, a = scipy.signal.butter(order, [l, h], btype='bandpass')
    yf = scipy.signal.filtfilt(b, a, signal, axis=0)
    return yf

In [4]:
def workOnColumn(f, id, PersonCount , StimuliIndex):
    for i in range(len(Channel)):
        y = np.array(f[Channel[i]])  # Accessing each Channel.
        #y = preprocessing.minmax_scale(y)  # Scaling 
        cutTime = 0
        if id == 'Stimuli':
            cutTime = len(y) - 7680  # For Last 60 Seconds
        if id == 'Baseline':
            cutTime = len(y) - 512  # For Last 4 Seconds

        Filtered_Y = bandPassFilter(y[cutTime:], 4, 40)
        
        fs = 256
        N = len(np.array(Filtered_Y))
        amp =  amp = 2 * np.sqrt(2)

        s, t, Zxx = signal.stft(Filtered_Y, fs)
        plt.pcolormesh(t, s, np.abs(Zxx), vmin=0, vmax=amp, shading='gouraud')
        plt.axis('off')
        plt.gca().set_axis_off()
        plt.subplots_adjust(top = 1, bottom = 0, right = 1, left = 0, hspace = 0, wspace = 0)
        plt.margins(0,0)
        plt.gca().xaxis.set_major_locator(plt.NullLocator())
        plt.gca().yaxis.set_major_locator(plt.NullLocator())
        plt.savefig( 'Spectograms/' +'P'+str(PersonCount+1)+'_S'+ str(StimuliIndex+1)+'_'+ Channel[i] +'.jpg')
        


        

Extracting Channels from 414 stimuli data.

In [5]:
total_channels = 14
total_Person = 23 
total_Stimuli = 18

for PersonCount in range(total_Person):
    for index in range(total_Stimuli):
        # For Stimuli
        if(index>10):
            csvStimuli = pd.read_csv('Dataset/EEG_With_Channel/person_' + str(PersonCount + 1) + '_EEG_Stimuli_' + str(index + 1) + '.csv')
            Stimuli = pd.DataFrame(csvStimuli)
            workOnColumn(Stimuli, 'Stimuli', PersonCount , index)
    


KeyboardInterrupt: 